需要使用tensorflow1.12以上的版本

且不能使用tensorflow2

In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
os.getcwd()

'E:\\GitHub\\Text-Classification'

In [30]:
import collections
import tensorflow as tf
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
tf.__version__

'1.14.0'

In [3]:
from bert import tokenization

In [4]:
df = pd.read_csv('data/train.tsv', sep='\t')

In [5]:
df.loc[0,'content']

'下列关于“克隆羊”、“试管羊”、“转基因羊”的说法合理的是（）A它们的生殖方式相同B它们的遗传物质都只来自于一个亲本C它们的形成过程操作水平相同D在培育过程中都要运用胚胎移植技术题型:单选题|难度:一般|使用次数:0|答案：D解析：【解答】解：A、克隆动物采用了核移植技术，属于无性生殖，试管动物主要采用了体外受精技术，属于有性生殖，转基因动物是通过导入外源基因的方式来获取亲本的优良性状，A错误；B、克隆动物遗传物质分别来自提供细胞核和卵细胞质的亲本，试管动物的遗传物质来自有性生殖的两个亲本，转基因动物遗传物质来自有性生殖的两个亲本和提供目的基因的个体，可见，三种动物的遗传物质都不是只来自于一个亲本，B错误；C、“克隆羊”主要采用了核移植技术，属于细胞水平上的操作；“试管羊”主要采用的体外受精技术，属于胚胎水平上的操作；“转基因羊”主要采用了基因工程技术，属于分子水平上的操作，C错误；D、这三者在培育过程中都要运用胚胎移植技术，D正确．据此可知答案为：D．【考点精析】根据题目的已知条件，利用动物细胞核移植技术和胚胎移植的相关知识可以得到问题的答案，需要掌握动物体细胞核移植是将动物的一个细胞的细胞核，移入一个已经去掉细胞核的卵母细胞中，使其重组并发育成一个新的胚胎，这个新胚胎最终发育为动物个体；胚胎移植是将雌性动物的早期胚胎移植到同种的、生理状态相同的其他雌性动物的体内，使之继续发育为新个体的技术．'

In [6]:
# 保存label
label = set(' '.join(df['label']).strip().split())

with open('data/label.txt', 'w', encoding='utf-8') as f:
    for l in label:
        f.write(l+'\n')
len(label)

97

In [7]:
# 读取label
label = []
with open('data/label.txt', 'r', encoding='utf-8') as f:
    label = [l for l in f.read().strip().split('\n')]

In [8]:
print(label[:10])

['宇宙中的地球', '地球运动的基本形式', '地球的内部圈层结构及特点', '社会主义是中国人民的历史性选择', '减数分裂的概念', '基因的分离规律的实质及应用', '组成细胞的化合物', '基因工程的概念', '人体水盐平衡调节', '拉马克的进化学说']


# 加载数据

## 输入实例

一个InputExample实例如字面意思，包含了4个属性。

guid 是表示这格实例属于训练集、验证集或测试集的第几个样本：如`'train-34'`, `'dev-28'`

In [9]:
class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, text_b=None, label=None):
    """Constructs a InputExample.

    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label

## 数据处理基类

read_tsv 就是以\t分割的数据。在保存数据的时候指定一下分割符即可

In [10]:
class DataProcessor(object):
  """Base class for data converters for sequence classification data sets."""

  def get_train_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the train set."""
    raise NotImplementedError()

  def get_dev_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the dev set."""
    raise NotImplementedError()

  def get_test_examples(self, data_dir):
    """Gets a collection of `InputExample`s for prediction."""
    raise NotImplementedError()

  def get_labels(self):
    """Gets the list of labels for this data set."""
    raise NotImplementedError()

  @classmethod
  def _read_tsv(cls, input_file, quotechar=None):
    """Reads a tab separated value file."""
    with tf.gfile.Open(input_file, "r") as f:
      reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
      lines = []
      for line in reader:
        lines.append(line)
      return lines

In [11]:
class BaiduProcessor(DataProcessor):
  """Processor for the CoLA data set (GLUE version)."""

  def get_train_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

  def get_dev_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

  def get_test_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

  def get_labels(self):
    """See base class."""
    label_file = 'data/label.txt'  # 标签存放的位置
    labels = [l.strip() for l in open(label_file, encoding='utf-8').readlines()]
    return labels

  def _create_examples(self, lines, set_type):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, line) in enumerate(lines):
      if i == 0:
        continue
      guid = "%s-%s" % (set_type, i)
      text_a = tokenization.convert_to_unicode(line[1])
      text_b = None
      if set_type == "test":
        label = "0"
      else:
        label = tokenization.convert_to_unicode(line[0])
      examples.append(
          InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    return examples

## 我的数据处理类

In [12]:
processer = BaiduProcessor()
data_dir = 'data'

In [13]:
x = processer.get_dev_examples(data_dir)
x[0].label, x[0].guid, x[0].text_a

('政治 科学社会主义常识 社会主义是中国人民的历史性选择',
 'dev-1',
 '鱼传尺素、鸿雁传书、目断鲜鸿，这样的文化传统代代相因、世世相袭，融入百姓生活，升华为中华民族独特的文化现象一家书文化。据记者了解，2017年央视春晚将坚持“三不用原则”，即：低俗媚俗的节目不用，格调不高的节目不用，有污点和道德瑕疵的演员不用。坚持“三不用原则”，符合社会主义核心价值观的建设要求。之所以这样做是因为（）①优秀文化才对人具有潜移默化和深远持久的影响②必须加强对文化市场的管理和引导③广播电视成为低俗文化传播的主要手段④落后、腐朽文化严重腐蚀人们的精神世界A①②B①③C③④D②④题型:单选题|难度:一般|使用次数:0|答案：D解析：坚持“三不用原则”，即：低俗媚俗的节目不用，格调不高的节目不用，有污点和道德瑕疵的演员不用。坚持“三不用原则”，符合社会主义核心价值观的建设要求。之所以这样做是因为落后、腐朽文化严重腐蚀人们的精神世界，国家要加强管理、正确引导，②④适合题意；①说法正确，但不是本题的原因，排除；大众传媒是文化传播的主要手段，排除③。据此可知答案为：D。')

In [14]:
labels = processer.get_labels()

# 数据预处理部分
## convert_single_example

实现的过程是使：InputExample -> InputFeatures

input_ids : Token Embeddings<br>
segment_ids : Segment Embeddings<br>
label_id : 标签的类别<br>
- 如果是2分类或多分类，是int，如0 or 1 or 2 or .. 
- 如果是多分类，为列表如：`[0, 0, 1, 1, 0, 1]`

input_mask : 如果设定的长度比句子长度长，多出来的部分就要被mask掉。就是padding部分要被mask

![](../data/img/w13_bert_input.jpg)

In [15]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 input_ids,
                 input_mask,
                 segment_ids,
                 label_id,
                 is_real_example=True):
        self.input_ids = input_ids  # 
        self.input_mask = input_mask
        self.segment_ids = segment_ids # 
        self.label_id = label_id
        self.is_real_example = is_real_example

In [16]:
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.

    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.

    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """

In [17]:
def convert_single_example(ex_index, example, label_list, max_seq_length,
                                                     tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
                input_ids=[0] * max_seq_length,
                input_mask=[0] * max_seq_length,
                segment_ids=[0] * max_seq_length,
                label_id=0,
                is_real_example=False)

    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    label_id = example_label2id(example.label, label_map)  # 修改部分
    if ex_index < 5:
        tf.logging.info("*** Example ***")
        tf.logging.info("guid: %s" % (example.guid))
        tf.logging.info("tokens: %s" % " ".join(
                [tokenization.printable_text(x) for x in tokens]))
        tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
        tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
        tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
        # tf.logging.info("label: %s (id = %d)" % (example.label, label_id))  # 修改部分
        tf.logging.info("label_id: %s" % " ".join([str(x) for x in label_id]))  # 修改部分

    feature = InputFeatures(
            input_ids=input_ids,
            input_mask=input_mask,
            segment_ids=segment_ids,
            label_id=label_id,
            is_real_example=True)
    return feature

### _truncate_seq_pair

长句子截断，是tokens_a和b同时截断。且尽量保证两个句子一样长

In [18]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [19]:
tokens_a = ['你','好','世','界','！']
tokens_b = ['世', '界', '说', '：', '我', '不', '好', '。']
_truncate_seq_pair(tokens_a, tokens_b, 15)
tokens_a, tokens_b

(['你', '好', '世', '界', '！'], ['世', '界', '说', '：', '我', '不', '好', '。'])

In [20]:
tokens_a = ['你','好','世','界','！']
tokens_b = ['世', '界', '说', '：', '我', '不', '好', '。']
_truncate_seq_pair(tokens_a, tokens_b, 9)
tokens_a, tokens_b

(['你', '好', '世', '界', '！'], ['世', '界', '说', '：'])

### 修改label_id的生成方式

由于是多标签分类，需要修改label_id的生成方式

In [21]:
def example_label2id(label, label_map):
    """ 将样本标签转为数字 """
    n_class = len(label_map)  # 多标签的类别数
    label_id = [0] * n_class  # 初始化label_id为全0
    for lb in label.split():  # 根据样本的label，把对应位置标签置1
        label_id[label_map[lb]] = 1
    return label_id

In [22]:
label_map = {}
for (i, label) in enumerate(labels):
    label_map[label] = i
x[0].label

'政治 科学社会主义常识 社会主义是中国人民的历史性选择'

In [23]:
print(example_label2id(x[0].label, label_map))

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### 构建测试样例

In [24]:
ex_index = 1
example = x[2]
max_seq_length = 50
vocab_file = 'bert/pretrained_model/chinese_L-12_H-768_A-12/vocab.txt'
tokenizer = tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=False)

In [26]:
f = convert_single_example(ex_index, example, labels, max_seq_length, tokenizer)

INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: dev-3
INFO:tensorflow:tokens: [CLS] 噬 菌 体 、 细 菌 和 酵 母 菌 都 具 有 的 物 质 或 结 构 是 （ ） [UNK] . 细 胞 壁 [UNK] . 细 胞 质 [UNK] . 细 胞 膜 [UNK] . 遗 传 物 质 题 型 : 单 选 [SEP]
INFO:tensorflow:input_ids: 101 1693 5826 860 510 5301 5826 1469 6997 3678 5826 6963 1072 3300 4638 4289 6574 2772 5310 3354 3221 8020 8021 100 119 5301 5528 1880 100 119 5301 5528 6574 100 119 5301 5528 5606 100 119 6890 837 4289 6574 7579 1798 131 1296 6848 102
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label_id: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0


## file_based_convert_examples_to_features

将InputFeature保存为TFrecords 数据格式

https://blog.csdn.net/weixin_42001089/article/details/90236241

In [27]:
def file_based_convert_examples_to_features(
        examples, label_list, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example, label_list,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["label_ids"] = create_int_feature(feature.label_id)  # 修改 : 把中括号去掉了
        features["is_real_example"] = create_int_feature(
                [int(feature.is_real_example)])

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()

### 创建tf_example

In [28]:
def create_int_feature(values):
    f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
    return f


In [31]:
feature = f
features = collections.OrderedDict()
features["input_ids"] = create_int_feature(feature.input_ids)
features["input_mask"] = create_int_feature(feature.input_mask)
features["segment_ids"] = create_int_feature(feature.segment_ids)
features["label_ids"] = create_int_feature(feature.label_id)  # 修改 : 把中括号去掉了
features["is_real_example"] = create_int_feature(
        [int(feature.is_real_example)])

tf_example = tf.train.Example(features=tf.train.Features(feature=features))

### 测试

In [32]:
dev_file = 'bert/tmp/dev.tf_record'
examples = processer.get_dev_examples(data_dir)
label_list = processer.get_labels()
max_seq_length = 128
tokenizer = tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=False)
# output_file = dev_file

In [ ]:
file_based_convert_examples_to_features(
                examples, label_list, max_seq_length, tokenizer, dev_file)